In [6]:
import polars as pl
import pandas as pd
import plotly.express as px

# Profil Poktan

In [63]:
bkb = pl.read_excel("data/data_bkb.xlsx")
bkr = pl.read_excel("data/data_bkr.xlsx")
bkl = pl.read_excel("data/data_bkl.xlsx")
uppka = pl.read_excel("data/data_uppka.xlsx")
pikr = pl.read_excel("data/data_pikr.xlsx")
kkb = pl.read_excel("data/data_kkb.xlsx")
rdk = pl.read_excel("data/data_rdk.xlsx")
daftar_desa = pl.read_csv("data/data_daftar_desa.csv")

In [7]:
filter_kabupaten = ["POLEWALI MANDAR", "POLEWALI MANDAR"]
filter_kecamatan = ["MAPILLI", "MAPILLI"]
filter_desa = ["MAPILI", "KURMA"]
filter_bulan = ["JUNI"]


def filter_poktan(data, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan):
    hasil = data.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa),
        pl.col("BULAN").is_in(filter_bulan)
    )
    hasil = hasil.drop("BATAS")

    return hasil

In [51]:
bkb = filter_poktan(bkb, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
bkr = filter_poktan(bkr, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
bkl = filter_poktan(bkl, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
uppka = filter_poktan(uppka, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
pikr = filter_poktan(pikr, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
kkb = filter_poktan(kkb, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
rdk = filter_poktan(rdk, filter_kabupaten, filter_kecamatan, filter_desa, filter_bulan)
daftar_desa = daftar_desa.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa)
    )

In [55]:
data_poktan = pl.DataFrame({
    "Desa/Kel": daftar_desa.height,
    "Kampung KB": kkb["JUMLAH_KKB"].sum(),
    "Rumah DataKu": rdk["JUMLAH_RDK"].sum(),
    "BKB": bkr["JUMLAH_BKR"].sum(),
    "BKR": bkr["JUMLAH_BKR"].sum(),
    "BKL": bkl["JUMLAH_BKL"].sum(),
    "UPPKA": uppka["JUMLAH_UPPKA"].sum(),
    "PIK-R": pikr["JUMLAH_PIKR"].sum()
})

data_poktan

Desa/Kel,Kampung KB,Rumah DataKu,BKB,BKR,BKL,UPPKA,PIK-R
i64,i64,i64,i64,i64,i64,i64,i64
2,2,2,2,2,3,0,2


In [56]:
data_poktan.melt(value_vars=["Desa/Kel", "Kampung KB", "Rumah DataKu",
                            "BKB",	"BKR",	"BKL",	"UPPKA",	"PIK-R"], 
                            variable_name="Variable", value_name="Value")

/tmp/ipykernel_3417/4204789583.py:1: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_poktan.melt(value_vars=["Desa/Kel", "Kampung KB", "Rumah DataKu",


Variable,Value
str,i64
"""Desa/Kel""",2
"""Kampung KB""",2
"""Rumah DataKu""",2
"""BKB""",2
"""BKR""",2
"""BKL""",3
"""UPPKA""",0
"""PIK-R""",2


In [58]:
data_poktan.unpivot(["Desa/Kel", "Kampung KB", "Rumah DataKu",
                            "BKB",	"BKR",	"BKL",	"UPPKA",	"PIK-R"], 
                            variable_name="Variable", value_name="Value")

Variable,Value
str,i64
"""Desa/Kel""",2
"""Kampung KB""",2
"""Rumah DataKu""",2
"""BKB""",2
"""BKR""",2
"""BKL""",3
"""UPPKA""",0
"""PIK-R""",2


### KB

In [3]:
from shiny import App, Inputs, Outputs, Session, reactive, render, ui

def nilai_bulan_sebelum(bulan_terpilih):
    daftar_bulan = ["JANUARI", "FEBRUARI", "MARET", "APRIL", "MEI", "JUNI", "JULI", "AGUSTUS", "SEPTEMBER", "OKTOBER", "NOVEMBER", "DESEMBER"]
    index = daftar_bulan.index(bulan_terpilih)
    if index == 0:
        return daftar_bulan[0]  # Jika bulan terpilih adalah JANUARI, nilai_bulan_sebelum juga JANUARI
    else:
        return daftar_bulan[index - 1]

In [4]:
data_mix = pl.read_excel("data/data_mix_kontra.xlsx")


data_mix = data_mix.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(filter_bulan)
)
data_mix



__UNNAMED__0,Provinsi,Kabupaten,Kecamatan,Kelurahan/Desa,PA,Suntik,Pil,Kondom,Implan,IUD,Vasektomi,Tubektomi,MAL,KB Modern,KB Tradisional,Bulan
i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
3484,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""",77,35,5,2,23,0,0,12,0,77,0,"""JUNI"""
3487,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""",242,141,30,0,56,1,0,14,0,242,0,"""JUNI"""


In [12]:

data_pus = pl.read_excel("data/data_pus.xlsx")

data_pus = data_pus.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(filter_bulan)
)
unmet_need_bulan_ini = round(data_pus["Unmet Need"].sum() / data_pus["PUS"].sum() *100, 2)
unmet_need_bulan_ini

27.86

In [13]:

data_pus_bulan_lalu = pl.read_excel("data/data_pus.xlsx")

data_pus_bulan_lalu = data_pus_bulan_lalu.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in([nilai_bulan_sebelum("JUNI")])
)
unmet_need_bulan_lalu = round(data_pus_bulan_lalu["Unmet Need"].sum() / data_pus_bulan_lalu["PUS"].sum() *100, 2)
unmet_need_bulan_lalu

27.96

In [18]:
"a" + "b"

'ab'

In [ ]:
if unmet_need_bulan_ini > unmet_need_bulan_lalu:
    icon_title = "Unmet Need"
    icons_fa = ui.HTML(
        f'<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 384 512"><!--!Font Awesome Free 6.5.2 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path d="M214.6 41.4c-12.5-12.5-32.8-12.5-45.3 0l-160 160c-12.5 12.5-12.5 32.8 0 45.3s32.8 12.5 45.3 0L160 141.2V448c0 17.7 14.3 32 32 32s32-14.3 32-32V141.2L329.4 246.6c12.5 12.5 32.8 12.5 45.3 0s12.5-32.8 0-45.3l-160-160z"/></svg>'   
    )
    kondisi = "Naik"
elif unmet_need_bulan_ini < unmet_need_bulan_lalu:
    icon_title = "Unmet Need"
    icons_fa = ui.HTML(
        f'<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 384 512"><!--!Font Awesome Free 6.5.2 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path d="M169.4 470.6c12.5 12.5 32.8 12.5 45.3 0l160-160c12.5-12.5 12.5-32.8 0-45.3s-32.8-12.5-45.3 0L224 370.8 224 64c0-17.7-14.3-32-32-32s-32 14.3-32 32l0 306.7L54.6 265.4c-12.5-12.5-32.8-12.5-45.3 0s-12.5 32.8 0 45.3l160 160z"/></svg>'
    )
    kondisi = "Turun"
else unmet_need_bulan_ini == unmet_need_bulan_lalu:
    icon_title = "Unmet Need"
    icons_fa = ui.HTML(
        f'<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 512 512"><!--!Font Awesome Free 6.5.2 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path d="M105.1 202.6c7.7-21.8 20.2-42.3 37.8-59.8c62.5-62.5 163.8-62.5 226.3 0L386.3 160H352c-17.7 0-32 14.3-32 32s14.3 32 32 32H463.5c0 0 0 0 0 0h.4c17.7 0 32-14.3 32-32V80c0-17.7-14.3-32-32-32s-32 14.3-32 32v35.2L414.4 97.6c-87.5-87.5-229.3-87.5-316.8 0C73.2 122 55.6 150.7 44.8 181.4c-5.9 16.7 2.9 34.9 19.5 40.8s34.9-2.9 40.8-19.5zM39 289.3c-5 1.5-9.8 4.2-13.7 8.2c-4 4-6.7 8.8-8.1 14c-.3 1.2-.6 2.5-.8 3.8c-.3 1.7-.4 3.4-.4 5.1V432c0 17.7 14.3 32 32 32s32-14.3 32-32V396.9l17.6 17.5 0 0c87.5 87.4 229.3 87.4 316.7 0c24.4-24.4 42.1-53.1 52.9-83.7c5.9-16.7-2.9-34.9-19.5-40.8s-34.9 2.9-40.8 19.5c-7.7 21.8-20.2 42.3-37.8 59.8c-62.5 62.5-163.8 62.5-226.3 0l-.1-.1L125.6 352H160c17.7 0 32-14.3 32-32s-14.3-32-32-32H48.4c-1.6 0-3.2 .1-4.8 .3s-3.1 .5-4.6 1z"/></svg>'
    )
    kondisi = "Sama"

In [3]:
data_pus_unmet_need = pl.read_excel("data/data_pus.xlsx")

def bulan_hingga(bulan_terpilih):
    daftar_bulan = ["JANUARI", "FEBRUARI", "MARET", "APRIL", "MEI", "JUNI", "JULI", "AGUSTUS", "SEPTEMBER", "OKTOBER", "NOVEMBER", "DESEMBER"]
    index = daftar_bulan.index(bulan_terpilih)
    return daftar_bulan[:index + 1]

hingga_bulan = bulan_hingga("MARET")

data_pus_unmet_need = data_pus_unmet_need.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(hingga_bulan)
)
data_pus_unmet_need = data_pus_unmet_need.group_by("Bulan").agg(
    [
        pl.sum("PUS").alias("PUS"),
        pl.sum("Unmet Need").alias("Unmet Need")
    ]
)

# Buat kolom baru yang merupakan hasil pembagian 'total_nilai1' dengan 'total_nilai2'
data_pus_unmet_need = data_pus_unmet_need.with_columns(
    (pl.col("Unmet Need") / pl.col("PUS") *100 ).alias("Unmet Need(%)")
)

data_pus_unmet_need = data_pus_unmet_need.with_columns(
    pl.col("Unmet Need(%)").round(2)
)

# Peta urutan bulan
bulan_urut = {
    "JANUARI": 1,
    "FEBRUARI": 2,
    "MARET": 3,
    "APRIL": 4,
    "MEI": 5,
    "JUNI": 6,
    "JULI": 7,
    "AGUSTUS": 8,
    "SEPTEMBER": 9,
    "OKTOBER": 10,
    "NOVEMBER": 11,
    "DESEMBER": 12
}

data_pus_unmet_need


Bulan,PUS,Unmet Need,Unmet Need(%)
str,i64,i64,f64
"""MARET""",775,234,30.19
"""JANUARI""",778,236,30.33
"""FEBRUARI""",775,235,30.32


In [25]:
import plotly.express as px
# Konversi ke pandas DataFrame untuk kompatibilitas dengan Plotly
df_pandas = data_pus_unmet_need.to_pandas()


# Peta urutan bulan
bulan_urut = {
    "JANUARI": 1,
    "FEBRUARI": 2,
    "MARET": 3,
    "APRIL": 4,
    "MEI": 5,
    "JUNI": 6,
    "JULI": 7,
    "AGUSTUS": 8,
    "SEPTEMBER": 9,
    "OKTOBER": 10,
    "NOVEMBER": 11,
    "DESEMBER": 12
}

# Tambahkan kolom urutan bulan di pandas DataFrame
df_pandas['Bulan_urut'] = df_pandas['Bulan'].map(bulan_urut)

# Urutkan DataFrame berdasarkan urutan bulan
df_pandas = df_pandas.sort_values(by='Bulan_urut')

# Buang kolom urutan bulan setelah pengurutan
df_pandas = df_pandas.drop(columns=['Bulan_urut'])


# Buat line chart menggunakan Plotly
fig = px.line(df_pandas, x="Bulan", y=["Unmet Need(%)"])
fig.update_traces(
    line_color="#406EF1",
    line_width=1,
    fill="tozeroy",
    fillcolor="rgba(64,110,241,0.2)",
    hoverinfo="y",
)
fig.update_xaxes(visible=False, showgrid=False)
fig.update_yaxes(visible=False, showgrid=False)
fig.update_layout(
    height=100,
    hovermode="x",
    margin=dict(t=0, r=0, l=0, b=0),
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
)

# Tampilkan chart
fig.show(config = {'displayModeBar': False})
#fig.show(config={'displayModeBar':False})

In [31]:
daftar_bulan = ["JANUARI", "FEBRUARI", "MARET", "APRIL", "MEI", "JUNI", "JULI", "AGUSTUS", "SEPTEMBER", "OKTOBER", "NOVEMBER", "DESEMBER"]
daftar_bulan[6]

'JULI'

In [41]:
data_tenakes = pl.read_excel("data/data_faskes_siga.xlsx")
data_tenakes = data_tenakes.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("BULAN").is_in(filter_bulan)
)
jumlah_tenakes = data_tenakes.height

if jumlah_tenakes <= 0:
    jumlah_tenakes = 0
else:
    jumlah_tenakes = jumlah_tenakes

jumlah_tenakes



0

In [44]:
data_tp_kb = pl.read_excel("data/data_faskes_siga.xlsx")
data_tp_kb = data_tp_kb.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("BULAN").is_in(filter_bulan)
)
len(data_tp_kb["No. Registrasi"].unique())


__UNNAMED__0,Provinsi,Kabupaten,Kecamatan,Kelurahan/Desa,Nama Faskes,No. Registrasi,Nama Bidan,NIK Bidan,Alamat,No HP,Profesi,Pelatihan,Status Update,BULAN,BATAS
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
1524,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""","""Puskesmas Mapilli""","""7604028""","""Hernawati, A.Md. Keb""","""7604084609870007""","""MAPILLI""","""""","""Bidan""","""IUD/IMPLAN,R/R,Sertifikat Komp…","""UPDATE""","""JUNI""",null
1525,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""","""Puskesmas Mapilli""","""7604028""","""IRAWAHYUNI RASYID,S.ST""","""7604044505880003""","""PEKKABATA""","""""","""Bidan""","""IUD/IMPLAN,Sertifikat Kompeten…","""UPDATE""","""JUNI""",null
1540,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""","""POSKESDES KURMA""","""760402810""","""HASNAWATI. M""","""7315056403790003""","""DUSUN PARADEANG""","""085394047037""","""Bidan""","""Belum Terlatih""","""UPDATE""","""JUNI""",null


In [35]:
# filter_kabupaten = val_kab.get()
# filter_kecamatan = val_kec.get()
# filter_desa = val_desa.get()
# filter_bulan = input.pilih_bulan()
data_mkjp = pl.read_excel("data/data_mix_kontra.xlsx")

data_mkjp = data_mkjp.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(["JUNI"])
)
mkjp_bulan_ini = data_mkjp["Implan"].sum() + data_mkjp["IUD"].sum() + data_mkjp["Vasektomi"].sum() + data_mkjp["Tubektomi"].sum()	
#unmet_need_bulan_ini

data_mkjp_bulan_lalu = pl.read_excel("data/data_mix_kontra.xlsx")

data_mkjp_bulan_lalu = data_mkjp_bulan_lalu.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in([nilai_bulan_sebelum("JUNI")])
)
mkjp_bulan_lalu = data_mkjp_bulan_lalu["Implan"].sum() + data_mkjp_bulan_lalu["IUD"].sum() + data_mkjp_bulan_lalu["Vasektomi"].sum() + data_mkjp_bulan_lalu["Tubektomi"].sum()
mkjp_bulan_lalu


if mkjp_bulan_ini > mkjp_bulan_lalu:
    icon_title = "MKJP"
    icons_fa = ui.HTML(
        f'<svg aria-hidden="true" role="img" viewBox="0 0 384 512" style="height:1em;width:0.75em;vertical-align:-0.125em;margin-left:auto;margin-right:auto;font-size:inherit;fill:currentColor;overflow:visible;position:relative;"><path d="M214.6 41.4c-12.5-12.5-32.8-12.5-45.3 0l-160 160c-12.5 12.5-12.5 32.8 0 45.3s32.8 12.5 45.3 0L160 141.2V448c0 17.7 14.3 32 32 32s32-14.3 32-32V141.2L329.4 246.6c12.5 12.5 32.8 12.5 45.3 0s12.5-32.8 0-45.3l-160-160z"/></svg>'
        )
    kondisi = "Naik"
elif mkjp_bulan_ini < mkjp_bulan_lalu:
    icon_title = "MKJP"
    icons_fa = ui.HTML(
        f'<svg aria-hidden="true" role="img" viewBox="0 0 384 512" style="height:1em;width:0.75em;vertical-align:-0.125em;margin-left:auto;margin-right:auto;font-size:inherit;fill:currentColor;overflow:visible;position:relative;"><path d="M169.4 470.6c12.5 12.5 32.8 12.5 45.3 0l160-160c12.5-12.5 12.5-32.8 0-45.3s-32.8-12.5-45.3 0L224 370.8 224 64c0-17.7-14.3-32-32-32s-32 14.3-32 32l0 306.7L54.6 265.4c-12.5-12.5-32.8-12.5-45.3 0s-12.5 32.8 0 45.3l160 160z"/></svg>'
        )
    kondisi = "Turun"
else:
    icon_title = "MKJP"
    icons_fa = ui.HTML(
        f'<svg aria-hidden="true" role="img" viewBox="0 0 512 512" style="height:1em;width:1em;vertical-align:-0.125em;margin-left:auto;margin-right:auto;font-size:inherit;fill:currentColor;overflow:visible;position:relative;"><path d="M105.1 202.6c7.7-21.8 20.2-42.3 37.8-59.8c62.5-62.5 163.8-62.5 226.3 0L386.3 160H336c-17.7 0-32 14.3-32 32s14.3 32 32 32H463.5c0 0 0 0 0 0h.4c17.7 0 32-14.3 32-32V64c0-17.7-14.3-32-32-32s-32 14.3-32 32v51.2L414.4 97.6c-87.5-87.5-229.3-87.5-316.8 0C73.2 122 55.6 150.7 44.8 181.4c-5.9 16.7 2.9 34.9 19.5 40.8s34.9-2.9 40.8-19.5zM39 289.3c-5 1.5-9.8 4.2-13.7 8.2c-4 4-6.7 8.8-8.1 14c-.3 1.2-.6 2.5-.8 3.8c-.3 1.7-.4 3.4-.4 5.1V448c0 17.7 14.3 32 32 32s32-14.3 32-32V396.9l17.6 17.5 0 0c87.5 87.4 229.3 87.4 316.7 0c24.4-24.4 42.1-53.1 52.9-83.7c5.9-16.7-2.9-34.9-19.5-40.8s-34.9 2.9-40.8 19.5c-7.7 21.8-20.2 42.3-37.8 59.8c-62.5 62.5-163.8 62.5-226.3 0l-.1-.1L125.6 352H176c17.7 0 32-14.3 32-32s-14.3-32-32-32H48.4c-1.6 0-3.2 .1-4.8 .3s-3.1 .5-4.6 1z"/></svg>'
    )
    kondisi = "Sama"

icon_vb = ui.HTML(
        '<svg xmlns="http://www.w3.org/2000/svg" color="white" viewBox="0 0 384 512"><!--!Font Awesome Free 6.5.2 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path d="M192 0a48 48 0 1 1 0 96 48 48 0 1 1 0-96zM120 383c-13.8-3.6-24-16.1-24-31V296.9l-4.6 7.6c-9.1 15.1-28.8 20-43.9 10.9s-20-28.8-10.9-43.9l58.3-97c15-24.9 40.3-41.5 68.7-45.6c4.1-.6 8.2-1 12.5-1h1.1 12.5H192c1.4 0 2.8 .1 4.1 .3c35.7 2.9 65.4 29.3 72.1 65l6.1 32.5c44.3 8.6 77.7 47.5 77.7 94.3v32c0 17.7-14.3 32-32 32H304 264v96c0 17.7-14.3 32-32 32s-32-14.3-32-32V384h-8-8v96c0 17.7-14.3 32-32 32s-32-14.3-32-32V383z"/></svg>'
    )

#icon_title_tes = "About tooltips"
# icons_fa = ui.HTML(
#     f'<svg aria-hidden="true" color="blue" role="img" viewBox="0 0 512 512" style="height:1em;width:1em;vertical-align:-0.125em;margin-left:auto;margin-right:auto;font-size:inherit;fill:currentColor;overflow:visible;position:relative;"><title>{icon_title}</title><path d="M256 512A256 256 0 1 0 256 0a256 256 0 1 0 0 512zM216 336h24V272H216c-13.3 0-24-10.7-24-24s10.7-24 24-24h48c13.3 0 24 10.7 24 24v88h8c13.3 0 24 10.7 24 24s-10.7 24-24 24H216c-13.3 0-24-10.7-24-24s10.7-24 24-24zm40-208a32 32 0 1 1 0 64 32 32 0 1 1 0-64z"/></svg>'
# )
def fa_info_circle(icon_title: str, icons_fa):
    # Enhanced from https://rstudio.github.io/fontawesome/ via `fontawesome::fa(&quot;info-circle&quot;, a11y = &quot;sem&quot;, title = icon_title)`
    return ui.span(icon_title, " ", icons_fa)

judul_vb = fa_info_circle(icon_title, icons_fa)

if kondisi == "Naik":
    penjelasan_mkjp = "Naik dari capaian " + nilai_bulan_sebelum("JUNI") + " (" + str(mkjp_bulan_lalu)  + ")"
elif kondisi == "Turun":
        penjelasan_mkjp ="Turun dari capaian " + nilai_bulan_sebelum("JUNI") + " (" + str(mkjp_bulan_lalu)  + ")"
else:
    penjelasan_mkjp = "Sama dengan capaian " + nilai_bulan_sebelum("JUNI") + " (" + str(mkjp_bulan_lalu)  + ")"
import faicons
vb = ui.value_box(  
    ui.span(judul_vb, style="font-size:20px; font-weight: bold;"),
    mkjp_bulan_ini,
    penjelasan_mkjp,
    showcase= faicons.icon_svg("person-pregnant"),
    theme=ui.ValueBoxTheme(class_="", bg = "#f6f8fa", fg = "#0B538E"),
    full_screen=True,
)


<div id="bslib_card_8ad70d" class="card bslib-card bslib-mb-spacing bslib-value-box showcase-left-center html-fill-item html-fill-container" data-bslib-card-init="" data-full-screen="false" style="color:#0B538E;background-color:#f6f8fa;--bslib-color-fg:#0B538E;--bslib-color-bg:#f6f8fa;">
  <div class="card-body bslib-gap-spacing html-fill-item html-fill-container" style="margin-top:auto;margin-bottom:auto;flex:1 1 auto; padding:0;">
    <div class="value-box-grid html-fill-item" style="--bslib-grid-height:auto;--bslib-grid-height-mobile:auto;---bslib-value-box-showcase-w:30%;---bslib-value-box-showcase-w-fs:1fr;---bslib-value-box-showcase-max-h:100px;---bslib-value-box-showcase-max-h-fs:67%;">
      <div class="value-box-showcase html-fill-item html-fill-container">
        <svg viewBox="0 0 384 512" preserveAspectRatio="none" aria-hidden="true" role="img" class="fa" style="fill:currentColor;height:1em;width:0.75em;margin-left:auto;margin-right:0.2em;position:relative;vertical-align:-0.125em;overflow:visible;">          <path d="M192 96c-26.5 0-48-21.5-48-48s21.5-48 48-48s48 21.5 48 48s-21.5 48-48 48zM120 383c-13.8-3.6-24-16.1-24-31V296.9l-4.6 7.6c-9.1 15.1-28.8 20-43.9 10.9s-20-28.8-10.9-43.9l58.3-97c15-24.9 40.3-41.5 68.7-45.6c4.1-.6 8.2-1 12.5-1h1.1 12.5H192c1.4 0 2.8 .1 4.1 .3c35.7 2.9 65.4 29.3 72.1 65l6.1 32.5c44.3 8.6 77.7 47.5 77.7 94.3v32c0 17.7-14.3 32-32 32H304 264v96c0 17.7-14.3 32-32 32s-32-14.3-32-32V384h-8-8v96c0 17.7-14.3 32-32 32s-32-14.3-32-32V383z"></path></svg>
      </div>
      <div class="value-box-area html-fill-item html-fill-container">
        <div class="value-box-title html-fill-item html-fill-container">
          <span style="font-size:20px; font-weight: bold;"><span>MKJP <svg aria-hidden="true" role="img" viewBox="0 0 384 512" style="height:1em;width:0.75em;vertical-align:-0.125em;margin-left:auto;margin-right:auto;font-size:inherit;fill:currentColor;overflow:visible;position:relative;"><path d="M214.6 41.4c-12.5-12.5-32.8-12.5-45.3 0l-160 160c-12.5 12.5-12.5 32.8 0 45.3s32.8 12.5 45.3 0L160 141.2V448c0 17.7 14.3 32 32 32s32-14.3 32-32V141.2L329.4 246.6c12.5 12.5 32.8 12.5 45.3 0s12.5-32.8 0-45.3l-160-160z"/></svg></span></span>
        </div>
        <div class="value-box-value html-fill-item html-fill-container">
          <p>106</p>
        </div>
        Naik dari capaian MEI (98)
      </div>
    </div>
  </div>
  <bslib-tooltip placement="auto"><template style="display:none;">Expand</template><button class="bslib-full-screen-enter badge rounded-pill" aria-expanded="false" aria-controls="bslib_card_8ad70d" aria-label="Expand card"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" style="height:1em;width:1em;fill:currentColor;" aria-hidden="true" role="img"><path d="M20 5C20 4.4 19.6 4 19 4H13C12.4 4 12 3.6 12 3C12 2.4 12.4 2 13 2H21C21.6 2 22 2.4 22 3V11C22 11.6 21.6 12 21 12C20.4 12 20 11.6 20 11V5ZM4 19C4 19.6 4.4 20 5 20H11C11.6 20 12 20.4 12 21C12 21.6 11.6 22 11 22H3C2.4 22 2 21.6 2 21V13C2 12.4 2.4 12 3 12C3.6 12 4 12.4 4 13V19Z"/></svg></button></bslib-tooltip>
  <script data-bslib-card-init="">window.bslib.Card.initializeAllCards();</script>
</div>

In [12]:
# filter_kabupaten = val_kab.get()
# filter_kecamatan = val_kec.get()
# filter_desa = val_desa.get()
# filter_bulan = input.pilih_bulan()
data_mkjp = pl.read_excel("data/data_mix_kontra.xlsx")
data_mkjp = data_mkjp.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(["MARET", "JUNI"])
)
data_mkjp 

# total_all_years = data.select(pl.col("year_2019").sum(), pl.col("year_2020").sum())

data_mkjp = data_mkjp.select(
        pl.col("Suntik").sum(),
        pl.col("Pil").sum(),
        pl.col("Kondom").sum(),
        pl.col("Implan").sum(),
        pl.col("IUD").sum(),
        pl.col("Vasektomi").sum(),
        pl.col("Tubektomi").sum(),
        pl.col("MAL").sum()
)

data_mkjp = data_mkjp.unpivot(on = ["Suntik", "Pil", "Kondom",
                "Implan",	"IUD",	"Vasektomi",
                "Tubektomi",	"MAL"], 
                variable_name="Alokon", value_name="Total")
data_mkjp

Alokon,Total
str,i64
"""Suntik""",339
"""Pil""",68
"""Kondom""",4
"""Implan""",150
"""IUD""",2
"""Vasektomi""",0
"""Tubektomi""",44
"""MAL""",0


In [14]:
import plotly.express as px
import pandas as pd
# Membuat DataFrame
df = data_mkjp.to_pandas()

# Mengurutkan data berdasarkan nilai terkecil ke terbesar
df = df.sort_values(by="Total")

# # Membuat bar chart dengan label
# fig = px.bar(df, x="Variable", y="Value", title="Bar Chart", labels={"Variable": "Metode Kontrasepsi", "Value": "Jumlah"}, text="Value")

grafik_kontrasepsi = px.bar(
    df,
    x="Alokon",
    y="Total",
    text="Total",
    title="Penggunaan Metode Kontrasepsi"
)

# Menambahkan pengaturan layout
grafik_kontrasepsi.update_traces(marker_color='#0d6efd')
grafik_kontrasepsi.update_layout(
    xaxis_title="Metode Kontrasepsi",
    yaxis_title="Jumlah Penggunaan",
    xaxis=dict(categoryorder="total ascending"),
    font=dict(family="Arial"),
    margin=dict(l=50, r=50, b=50, t=50),
    paper_bgcolor="#f6f8fa",
    plot_bgcolor="#f6f8fa",
    hoverlabel=dict(
        bgcolor="white",
        font=dict(family="Arial")
    ),
    legend=dict(font=dict(family="Arial")),
    hovermode="closest",
    hoverdistance=30,
    updatemenus=[dict(font=dict(family="Arial"))]
)

# Menampilkan grafik
grafik_kontrasepsi.show()


In [8]:
# bidang terlatih

import polars as pl
data_bidan = pl.read_excel("data/data_faskes_siga.xlsx")

data_bidan = data_bidan.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("BULAN").is_in(filter_bulan)
)


data_bidan = data_bidan['Pelatihan']

df = pl.DataFrame(data_bidan)

# Konversi Polars DataFrame ke Pandas DataFrame
df_pd = df.to_pandas()

# Menambahkan kolom "Status Pelatihan" berdasarkan kondisi pada kolom "Pelatihan"
df_pd["Status Pelatihan"] = df_pd["Pelatihan"].apply(
    lambda x: "Terlatih" if ("IUD" in x or "IMPLAN" in x) else "Tidak Terlatih"
)

df_pd

,Pelatihan,Status Pelatihan
0,"IUD/IMPLAN,R/R,Sertifikat Kompetensi IUD/Implan",Terlatih
1,"IUD/IMPLAN,Sertifikat Kompetensi IUD/Implan",Terlatih
2,Belum Terlatih,Tidak Terlatih


In [6]:
import plotly.express as px
# Hitung jumlah masing-masing kategori
count = df_pd["Status Pelatihan"].value_counts()
count

Status Pelatihan
Terlatih          2
Tidak Terlatih    1
Name: count, dtype: int64

In [7]:

# Plot donut chart menggunakan Plotly Express
donut_status_pelatihan = px.pie(count, names=count.index, values=count.values,
             color=count.index, color_discrete_map={'Tidak Terlatih': '#ffc107', 'Terlatih': '#0d6efd'},
             hole=0.5, title='Persentase Status Pelatihan')

donut_status_pelatihan.update_layout(
    xaxis_title="Metode Kontrasepsi",
    yaxis_title="Jumlah Penggunaan",
    xaxis=dict(categoryorder="total ascending"),
    font=dict(family="Arial"),
    margin=dict(l=50, r=50, b=50, t=50),
    paper_bgcolor="#f6f8fa",
    plot_bgcolor="#f6f8fa",
    hoverlabel=dict(
        bgcolor="white",
        font=dict(family="Arial")
    ),
    legend=dict(font=dict(family="Arial")),
    hovermode="closest",
    hoverdistance=30,
    updatemenus=[dict(font=dict(family="Arial"))]
)

# Menampilkan plot
donut_status_pelatihan.show()

In [7]:
data_pa = pl.read_excel("data/data_mix_kontra.xlsx")
data_pa = data_pa.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(["MARET", "JUNI"])
)
data_pa = data_pa.select(["PA", "Bulan"])
data_pa = data_pa.group_by("Bulan").agg([
    pl.sum("PA")
])
data_pa

Bulan,PA
str,i64
"""MARET""",288
"""JUNI""",319


In [13]:
line_pa = px.line(data_pa, x="Bulan", y="PA", title="Tren Peserta KB Aktif", markers=True)



line_pa.update_layout(
    xaxis_title="Metode Kontrasepsi",
    yaxis_title="Jumlah Penggunaan",
    xaxis=dict(categoryorder="total ascending"),
    font=dict(family="Arial"),
    margin=dict(l=50, r=50, b=50, t=50),
    paper_bgcolor="#f6f8fa",
    plot_bgcolor="#f6f8fa",
    hoverlabel=dict(
        bgcolor="white",
        font=dict(family="Arial")
    ),
    legend=dict(font=dict(family="Arial")),
    hovermode="closest",
    hoverdistance=30,
    updatemenus=[dict(font=dict(family="Arial"))]
)
# Menampilkan chart
line_pa.show()

In [24]:
data_pus = pl.read_excel("data/data_pus.xlsx")

hingga_bulan = bulan_hingga("JULI")

data_pus = data_pus.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(hingga_bulan)
)
data_pus

__UNNAMED__0,Provinsi,Kabupaten,Kecamatan,Kelurahan_Desa,PUS,Unmet Need,Bulan
i64,str,str,str,str,i64,i64,str
96,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""",254,114,"""JANUARI"""
99,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""",524,122,"""JANUARI"""
789,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""",254,114,"""FEBRUARI"""
792,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""",521,121,"""FEBRUARI"""
1503,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""",256,114,"""MARET"""
…,…,…,…,…,…,…,…
2484,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""",527,120,"""APRIL"""
3009,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""",267,114,"""MEI"""
3012,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""",570,120,"""MEI"""


In [26]:
data_pus = data_pus.select(["PUS", "Bulan"])
data_pus = data_pus.group_by("Bulan").agg([
    pl.sum("PUS")
])
data_pus

Bulan,PUS
str,i64
"""MARET""",775
"""JANUARI""",778
"""JUNI""",840
"""MEI""",837
"""APRIL""",789
"""FEBRUARI""",775


In [27]:
data_pus = data_pus.to_pandas()

# Menentukan urutan bulan
bulan_order = ["JANUARI", "FEBRUARI", "MARET", "APRIL", "MEI", "JUNI", "JULI", "AGUSTUS", "SEPTEMBER", "OKTOBER", "NOVEMBER", "DESEMBER"]
data_pus['Bulan'] = pd.Categorical(data_pus['Bulan'], categories=bulan_order, ordered=True)

# Mengurutkan DataFrame berdasarkan urutan bulan
data_pus = data_pus.sort_values('Bulan')


In [28]:
data_pus

,Bulan,PUS
1,JANUARI,778
5,FEBRUARI,775
0,MARET,775
4,APRIL,789
3,MEI,837
2,JUNI,840


In [30]:

line_pa = px.line(data_pus, x="Bulan", y="PUS", title="Tren Peserta KB Aktif", markers=True)

line_pa.update_layout(
    xaxis_title="Metode Kontrasepsi",
    yaxis_title="Jumlah Penggunaan",
    #xaxis=dict(categoryorder="total ascending"),
    font=dict(family="Arial"),
    margin=dict(l=50, r=50, b=50, t=50),
    paper_bgcolor="#f6f8fa",
    plot_bgcolor="#f6f8fa",
    hoverlabel=dict(
        bgcolor="white",
        font=dict(family="Arial")
    ),
    legend=dict(font=dict(family="Arial")),
    hovermode="closest",
    hoverdistance=30,
    updatemenus=[dict(font=dict(family="Arial"))]
)

In [37]:
data_pus = pl.read_excel("data/data_pus.xlsx")

data_pus = data_pus.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(filter_bulan)
)
data_pus = data_pus['PUS'].sum()
data_pus

840

In [56]:
#MCPR
data_pus = pl.read_excel("data/data_pus.xlsx")

hingga_bulan = bulan_hingga("JUNI")

data_pus = data_pus.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(hingga_bulan)
)
data_pus = data_pus.select(["PUS", "Bulan"])
data_pus = data_pus.group_by("Bulan").agg([
    pl.sum("PUS")
])

#MCPR
data_mcpr = pl.read_excel("data/data_mix_kontra.xlsx")

hingga_bulan = bulan_hingga("JUNI")

data_mcpr = data_mcpr.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(hingga_bulan)
)
data_mcpr = data_mcpr.select(["KB Modern", "Bulan"])
data_mcpr = data_mcpr.group_by("Bulan").agg([
    pl.sum("KB Modern")
])

data_mcpr = data_mcpr.join(data_pus, on="Bulan", how="inner")

data_mcpr = data_mcpr.with_columns(
    (pl.col("KB Modern") / pl.col("PUS") * 100).round(2).alias("MCPR")
)
data_mcpr

Bulan,KB Modern,PUS,MCPR
str,i64,i64,f64
"""FEBRUARI""",283,775,36.52
"""APRIL""",299,789,37.9
"""MARET""",288,775,37.16
"""MEI""",307,837,36.68
"""JUNI""",319,840,37.98
"""JANUARI""",283,778,36.38


In [57]:
#MCPR
data_pus = pl.read_excel("data/data_pus.xlsx")

hingga_bulan = ["JUNI"]

data_pus = data_pus.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(hingga_bulan)
)
data_pus = data_pus.select(["PUS", "Bulan"])
data_pus = data_pus.group_by("Bulan").agg([
    pl.sum("PUS")
])

#MCPR
data_mcpr = pl.read_excel("data/data_mix_kontra.xlsx")

hingga_bulan = bulan_hingga("JUNI")

data_mcpr = data_mcpr.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(hingga_bulan)
)
data_mcpr = data_mcpr.select(["KB Modern", "Bulan"])
data_mcpr = data_mcpr.group_by("Bulan").agg([
    pl.sum("KB Modern")
])

data_mcpr = data_mcpr.join(data_pus, on="Bulan", how="inner")

data_mcpr = data_mcpr.with_columns(
    (pl.col("KB Modern") / pl.col("PUS") * 100).round(2).alias("MCPR")
)
data_mcpr

Bulan,KB Modern,PUS,MCPR
str,i64,i64,f64
"""JUNI""",319,840,37.98


In [63]:
#MCPR
data_pus = pl.read_excel("data/data_pus.xlsx")

data_pus = data_pus.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan_Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(filter_bulan)
)
data_pus = data_pus.select(["PUS", "Bulan"])
data_pus = data_pus.group_by("Bulan").agg([
    pl.sum("PUS")
])

#MCPR
data_mcpr = pl.read_excel("data/data_mix_kontra.xlsx")

data_mcpr = data_mcpr.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("Bulan").is_in(filter_bulan)
)
data_mcpr = data_mcpr.select(["KB Modern", "Bulan"])
data_mcpr = data_mcpr.group_by("Bulan").agg([
    pl.sum("KB Modern")
])

data_mcpr = data_mcpr.join(data_pus, on="Bulan", how="inner")

data_mcpr = data_mcpr.with_columns(
    (pl.col("KB Modern") / pl.col("PUS") * 100).round(2).alias("MCPR")
)

data_mcpr = data_mcpr["MCPR"].sum()
data_mcpr

37.98

Bulan,KB Modern,PUS,MCPR
str,i64,i64,f64
"""APRIL""",299,789,37.9
"""MARET""",288,775,37.16
"""FEBRUARI""",283,775,36.52
"""JUNI""",319,840,37.98
"""MEI""",307,837,36.68
"""JANUARI""",283,778,36.38


In [15]:
data_bidan = pl.read_excel("data/data_faskes_siga.xlsx")

data_bidan = data_bidan.filter(
    pl.col("Kabupaten").is_in(filter_kabupaten),
    pl.col("Kecamatan").is_in(filter_kecamatan),
    pl.col("Kelurahan/Desa").is_in(filter_desa),
    pl.col("BULAN").is_in(filter_bulan)
)			 		

data_bidan



__UNNAMED__0,Provinsi,Kabupaten,Kecamatan,Kelurahan/Desa,Nama Faskes,No. Registrasi,Nama Bidan,NIK Bidan,Alamat,No HP,Profesi,Pelatihan,Status Update,BULAN,BATAS
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
1524,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""","""Puskesmas Mapilli""","""7604028""","""Hernawati, A.Md. Keb""","""7604084609870007""","""MAPILLI""","""""","""Bidan""","""IUD/IMPLAN,R/R,Sertifikat Komp…","""UPDATE""","""JUNI""",null
1525,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""MAPILI""","""Puskesmas Mapilli""","""7604028""","""IRAWAHYUNI RASYID,S.ST""","""7604044505880003""","""PEKKABATA""","""""","""Bidan""","""IUD/IMPLAN,Sertifikat Kompeten…","""UPDATE""","""JUNI""",null
1540,"""SULAWESI BARAT""","""POLEWALI MANDAR""","""MAPILLI""","""KURMA""","""POSKESDES KURMA""","""760402810""","""HASNAWATI. M""","""7315056403790003""","""DUSUN PARADEANG""","""085394047037""","""Bidan""","""Belum Terlatih""","""UPDATE""","""JUNI""",null


In [18]:
data_bidan = data_bidan.select(
    pl.col("Provinsi"), pl.col("Kabupaten"),
    pl.col("Kecamatan"), pl.col("Kelurahan/Desa"), pl.col("Nama Faskes"),
    pl.col("Nama Bidan"), pl.col("Pelatihan")
)

df = pl.DataFrame(data_bidan)

# Konversi Polars DataFrame ke Pandas DataFrame
df_pd = df.to_pandas()

# Menambahkan kolom "Status Pelatihan" berdasarkan kondisi pada kolom "Pelatihan"
df_pd["Pelatihan"] = df_pd["Pelatihan"].apply(
    lambda x: "Terlatih" if ("IUD" in x or "IMPLAN" in x) else "Belum Terlatih"
)

# if input.pilih_kab() == "SEMUA KABUPATEN":
#     df_pd = df_pd.groupby(['Kabupaten', 'Pelatihan']).size().unstack(fill_value=0)
# elif input.pilih_kab() != "SEMUA KABUPATEN" and input.pilih_kec() == "SEMUA KECAMATAN":
#     df_pd = df_pd.groupby(['Provinsi', 'Kabupaten', 'Kecamatan', 'Pelatihan']).size().unstack(fill_value=0)
# elif input.pilih_kab() != "SEMUA KABUPATEN" and input.pilih_kec() != "SEMUA KECAMATAN" and input.pilih_desa() == "SEMUA DESA/KELURAHAN":
#     df_pd = df_pd.groupby(['Provinsi', 'Kabupaten', 'Kecamatan', "Kelurahan/Desa", 'Pelatihan']).size().unstack(fill_value=0)
# else:
#     df_pd = df_pd
df_pd.groupby(['Kabupaten', 'Kecamatan', 'Pelatihan']).size().unstack(fill_value=0)

,Pelatihan,Belum Terlatih,Terlatih
Kabupaten,Kecamatan,,
POLEWALI MANDAR,MAPILLI,1,2


In [24]:
import polars as pl

# Data sample
data = {
    'Provinsi': ['SULAWESI BARAT'] * 10,
    'Kabupaten': ['MAMASA'] * 10,
    'Kecamatan': ['TAWALIAN'] * 10,
    'Kelurahan/Desa': ['TAWALIAN'] * 10,
    'Nama Faskes': ['PUSKESMAS TAWALIAN'] * 10,
    'Nama Bidan': ['DEWI A.Md. KEB', 'FEDRIANA', 'LILINARWASTI', 'GLORIA', 'DIANA SULLE', 'NURNAINNAH', 'NUR AISYAH', 'ADEL B PASAMBOAN', 'WINIARTI', 'RATA SARI PATANDEAN'],
    'Pelatihan': ['Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih', 'Belum Terlatih']
}

# Membuat DataFrame Polars
df = pl.DataFrame(data)

# Mengelompokkan data dan menghitung jumlahnya
summary = df.group_by(['Kabupaten', 'Kecamatan', 'Kelurahan/Desa', 'Nama Faskes', 'Pelatihan']).agg(
    pl.col('Nama Bidan').count().alias('count')
)

# Mengubah format agar serupa dengan unstack di pandas
summary_pivot = summary.pivot(
    values='count',
    index=['Kabupaten', 'Kecamatan', 'Kelurahan/Desa', 'Nama Faskes'],
    on='Pelatihan'
).fill_null(0)

# Menampilkan hasil
print(summary_pivot)


shape: (1, 6)
┌───────────┬───────────┬────────────────┬────────────────────┬────────────────┬──────────┐
│ Kabupaten ┆ Kecamatan ┆ Kelurahan/Desa ┆ Nama Faskes        ┆ Belum Terlatih ┆ Terlatih │
│ ---       ┆ ---       ┆ ---            ┆ ---                ┆ ---            ┆ ---      │
│ str       ┆ str       ┆ str            ┆ str                ┆ u32            ┆ u32      │
╞═══════════╪═══════════╪════════════════╪════════════════════╪════════════════╪══════════╡
│ MAMASA    ┆ TAWALIAN  ┆ TAWALIAN       ┆ PUSKESMAS TAWALIAN ┆ 9              ┆ 1        │
└───────────┴───────────┴────────────────┴────────────────────┴────────────────┴──────────┘


/tmp/ipykernel_2922/1200261974.py:23: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  summary_pivot = summary.pivot(


## Batas bawah

In [2]:
data_poktan = pl.read_csv("data/profil_poktan.csv")
data_poktan.head(5)

PROVINSI,KABUPATEN,KECAMATAN,KELURAHAN,LATITUDE,LONGITUDE,PUS,Kampung KB,Rumah DataKU,BKB,BKR,BKL,UPPKA,PIK-R
str,str,str,str,f64,f64,i64,str,str,str,str,str,str,str
"""SULAWESI BARAT""","""MAMUJU TENGAH""","""TOBADAK""","""MAHAHE""",-2.122958,119.365442,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada"""
"""SULAWESI BARAT""","""MAMUJU TENGAH""","""TOBADAK""","""SALOBAJA""",-2.156691,119.336074,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada"""
"""SULAWESI BARAT""","""MAMUJU TENGAH""","""TOBADAK""","""TOBADAK""",-2.104302,119.491646,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Tidak ada"""
"""SULAWESI BARAT""","""MAMUJU TENGAH""","""TOBADAK""","""PALONGAAN""",-2.105264,119.40054,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Tidak ada""","""Tidak ada"""
"""SULAWESI BARAT""","""MAMUJU TENGAH""","""TOBADAK""","""BATU PARIGI""",-2.07222,119.538128,1,"""Ada""","""Ada""","""Ada""","""Tidak ada""","""Ada""","""Tidak ada""","""Ada"""


In [9]:
# List kolom yang akan dihitung
columns_to_count = ["Kampung KB", "Rumah DataKU", "BKB", "BKR", "BKL", "UPPKA", "PIK-R"]

# Menghitung jumlah "Ada" untuk setiap kolom
counts = [data_poktan[col].str.count_matches("Ada").sum() for col in columns_to_count]

# Menghitung jumlah desa unik berdasarkan kombinasi KECAMATAN dan KELURAHAN
unique_desa = data_poktan.select(["KECAMATAN", "KELURAHAN"]).unique().shape[0]

# Membuat DataFrame hasil
results = pl.DataFrame({
    "Variable": ["Desa/Kelurahan"] + columns_to_count,
    "jumlah_ada": [unique_desa] + counts
})

results

Variable,jumlah_ada
str,i64
"""Desa/Kelurahan…",648
"""Kampung KB""",518
"""Rumah DataKU""",506
"""BKB""",595
"""BKR""",505
"""BKL""",505
"""UPPKA""",430
"""PIK-R""",441


In [10]:
import polars as pl

# Membuat DataFrame dari data yang ada
data = {
    "KABUPATEN": ["MAMUJU TENGAH"] * 5,
    "KECAMATAN": ["TOBADAK"] * 5,
    "KELURAHAN": ["MAHAHE", "SALOBAJA", "TOBADAK", "PALONGAAN", "BATU PARIGI"],
    "LATITUDE": ["-2.122958", "-2.156691", "-2.104302", "-2.105264", "-2.07222"],
    "LONGITUDE": [119.365442, 119.336074, 119.491646, 119.40054, 119.538128],
    "PUS": [1] * 5,
    "Kampung KB": ["Ada"] * 5,
    "Rumah DataKU": ["Tidak ada"] * 5,
    "BKB": ["Ada"] * 5,
    "BKR": ["Ada"] * 5,
    "BKL": ["Tidak ada"] * 5,
    "UPPKA": ["Ada"] * 5,
    "PIK-R": ["Tidak ada"] * 5
}

df = pl.DataFrame(data)

# List kolom yang akan dihitung
columns_to_count = ["Kampung KB", "Rumah DataKU", "BKB", "BKR", "BKL", "UPPKA", "PIK-R"]

# Menghitung jumlah "Ada" untuk setiap kolom dan menyusunnya ke dalam DataFrame baru
results = pl.DataFrame({
    "Variable": columns_to_count,
    "jumlah_ada": [df[col].str.count_matches("Ada").sum() for col in columns_to_count]
})

# Menampilkan hasil
results


Variable,jumlah_ada
str,i64
"""Kampung KB""",5
"""Rumah DataKU""",0
"""BKB""",5
"""BKR""",5
"""BKL""",0
"""UPPKA""",5
"""PIK-R""",0


In [ ]:
import plotly.graph_objs as go
import pandas as pd

# Load the data into a pandas DataFrame
df = pd.read_csv("population_data.csv")

# Create the male and female bar traces
trace_male = go.Bar(x=df[df["gender"]=="M"]["count"],
                	y=df[df["gender"]=="M"]["age"],
                	orientation="h",
                	name="Male",
                	marker=dict(color="#1f77b4"))

trace_female = go.Bar(x=df[df["gender"]=="F"]["count"]*(-1),
                  	y=df[df["gender"]=="F"]["age"],
                  	orientation="h",
                  	name="Female",
                  	marker=dict(color="#d62728"))

# Create the layout
layout = go.Layout(title="Population Pyramid",
               	xaxis=dict(title="Count"),
               	yaxis=dict(title="Age"),
               	barmode="overlay",
               	bargap=0.1)

# Create the figure
fig = go.Figure(data=[trace_male, trace_female], layout=layout)

# Show the plot
fig.show()

In [52]:
kelompok_umur_lk = pl.read_csv('data/PIRAMIDA PENDUDUK - Laki-laki.csv')

# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_lk.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_lk.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_lk = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Laki-laki"
)

# Menambahkan kolom 'Jenis_Kelamin' dengan isi 'Laki-laki'
#melted_result_lk = melted_result.with_columns([pl.lit("Laki-laki").alias("Jenis_Kelamin")])

# Menampilkan hasil
melted_result_lk

PROVINSI,Age_Group,Laki-laki
str,str,i64
"""SULAWESI BARAT…","""Umur 0 - 1""",7655
"""SULAWESI BARAT…","""Umur 2 - 4""",31161
"""SULAWESI BARAT…","""Umur 5 - 9""",59619
"""SULAWESI BARAT…","""Umur 10 - 14""",62885
"""SULAWESI BARAT…","""Umur 15 - 19""",67733
"""SULAWESI BARAT…","""Umur 20 - 24""",63400
"""SULAWESI BARAT…","""Umur 25 - 29""",50836
"""SULAWESI BARAT…","""Umur 30 - 34""",44164
"""SULAWESI BARAT…","""Umur 35 - 39""",43901


In [53]:
kelompok_umur_lk.head(2)

PROVINSI,KABUPATEN,KECAMATAN,KODE,KELURAHAN,Jumlah_Penduduk,Umur 0 - 1,Umur 2 - 4,Umur 5 - 9,Umur 10 - 14,Umur 15 - 19,Umur 20 - 24,Umur 25 - 29,Umur 30 - 34,Umur 35 - 39,Umur 40 - 44,Umur 45 - 49,Umur 50 - 54,Umur 55 - 59,Umur 60 - 64,Umur 65 - 69,Umur 70 - 74,Umur 75+
str,str,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""SULAWESI BARAT…","""PASANGKAYU""","""BAMBALAMOTU""",1006,"""BAMBALAMOTU""",1994,52,111,205,217,263,186,157,129,142,135,107,115,76,46,19,20,14
"""SULAWESI BARAT…","""PASANGKAYU""","""BAMBALAMOTU""",2003,"""POLEWALI""",1712,20,96,188,172,213,166,125,107,126,125,107,100,58,53,21,21,14


In [40]:
kelompok_umur_pr = pl.read_csv('data/PIRAMIDA PENDUDUK - Perempuan.csv')
# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_pr.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_pr.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_pr = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Perempuan"
)

# Menambahkan kolom 'Jenis_Kelamin' dengan isi 'Perempuan'
#melted_result_pr = melted_result.with_columns([pl.lit("Perempuan").alias("Jenis_Kelamin")])

# Menampilkan hasil
melted_result_pr

PROVINSI,Age_Group,Perempuan
str,str,i64
"""SULAWESI BARAT…","""Umur 0 - 1""",7111
"""SULAWESI BARAT…","""Umur 2 - 4""",28925
"""SULAWESI BARAT…","""Umur 5 - 9""",55267
"""SULAWESI BARAT…","""Umur 10 - 14""",58918
"""SULAWESI BARAT…","""Umur 15 - 19""",62332
"""SULAWESI BARAT…","""Umur 20 - 24""",60634
"""SULAWESI BARAT…","""Umur 25 - 29""",49743
"""SULAWESI BARAT…","""Umur 30 - 34""",44449
"""SULAWESI BARAT…","""Umur 35 - 39""",44751


In [41]:
df_horizontal_join = melted_result_pr.join(melted_result_lk, on="Age_Group", how="inner")

# Daftar kategori usia
ku = ["0 - 1", "2 - 4", "5 - 9", "10 - 14", "15 - 19", 
      "20 - 24", "25 - 29", "30 - 34", "35 - 39", "40 - 44", 
      "45 - 49", "50 - 54", "55 - 59", "60 - 64", 
      "65 - 69", "70 - 74", "75+"]

# Menghitung berapa kali daftar ku perlu diulang
repeat_count = df_horizontal_join.shape[0] // len(ku) + 1
repeated_ku = (ku * repeat_count)[:df_horizontal_join.shape[0]]

# Menambahkan kolom kategori umur
df_horizontal_join = df_horizontal_join.with_columns([pl.Series(name="Kategori_Umur", values=repeated_ku)])

df_horizontal_join

PROVINSI,Age_Group,Perempuan,PROVINSI_right,Laki-laki,Kategori_Umur
str,str,i64,str,i64,str
"""SULAWESI BARAT…","""Umur 0 - 1""",7111,"""SULAWESI BARAT…",7655,"""0 - 1"""
"""SULAWESI BARAT…","""Umur 2 - 4""",28925,"""SULAWESI BARAT…",31161,"""2 - 4"""
"""SULAWESI BARAT…","""Umur 5 - 9""",55267,"""SULAWESI BARAT…",59619,"""5 - 9"""
"""SULAWESI BARAT…","""Umur 10 - 14""",58918,"""SULAWESI BARAT…",62885,"""10 - 14"""
"""SULAWESI BARAT…","""Umur 15 - 19""",62332,"""SULAWESI BARAT…",67733,"""15 - 19"""
"""SULAWESI BARAT…","""Umur 20 - 24""",60634,"""SULAWESI BARAT…",63400,"""20 - 24"""
"""SULAWESI BARAT…","""Umur 25 - 29""",49743,"""SULAWESI BARAT…",50836,"""25 - 29"""
"""SULAWESI BARAT…","""Umur 30 - 34""",44449,"""SULAWESI BARAT…",44164,"""30 - 34"""
"""SULAWESI BARAT…","""Umur 35 - 39""",44751,"""SULAWESI BARAT…",43901,"""35 - 39"""


In [43]:
y_age = df_horizontal_join['Kategori_Umur'] 
x_M = df_horizontal_join['Laki-laki'] 
x_F = df_horizontal_join['Perempuan'] * -1

if max(x_M) >= max(x_F):
    maks = max(x_M)
else:
    maks = max(x_F)

import math

def round_up_to_nearest(number, base):
    return base * math.ceil(number / base)

def auto_round_up(number):
    if number == 0:
        return 0
    base = 10 ** (len(str(number)) - 1)
    return round_up_to_nearest(number, base)

maks1 = auto_round_up(maks)
maks2 = auto_round_up(int(maks - (maks * 1 / 3)))
maks3 = auto_round_up(int(maks - (maks * 2 / 3)))

tick_vals = [-maks1, -maks2, -maks3, 0, maks1, maks2, maks3]
tick_str = [str(abs(value)) for value in tick_vals]

In [50]:
import plotly.graph_objects as gp
# Creating instance of the figure 
fig = gp.Figure() 
  

# Adding Female data to the figure 
fig.add_trace(gp.Bar(y = y_age, x = x_F, 
                     name = 'Perempuan', orientation = 'h',
                     marker=dict(color='#ffc107'),
                     hovertemplate='Perempuan Umur %{y}<br>Jumlah: %{customdata}<extra></extra>',
                     customdata=[abs(x) for x in x_F]
                     )) 

# Adding Male data to the figure 
fig.add_trace(gp.Bar(y= y_age, x = x_M,  
                     name = 'Laki-laki',  
                     orientation = 'h',
                     marker=dict(color='#0d6efd'),
                     hovertemplate='Laki-laki Umur %{y}<br> %{x}<extra></extra>')) 

  
# Updating the layout for our graph 
fig.update_layout(title = 'Piramida Penduduk', 
                 title_font_size = 22, barmode = 'relative', 
                 bargap = 0.0, bargroupgap = 0, 
                 xaxis = dict(tickvals = tick_vals, 
                              ticktext = tick_str, 
                              title = 'Jumlah', 
                              title_font_size = 14),
                legend=dict(
                        orientation='h',
                        yanchor='bottom',
                        y=-0.3,  # Adjust this value to move the legend up or down
                        xanchor='center',
                        x=0.5
                ),
                plot_bgcolor='#f6f8fa',  # Set background color of the plot area to green
                paper_bgcolor='#f6f8fa'  # Set background color of the entire canvas to green 
)
  
fig.show()

In [51]:
kelompok_umur_lk = pl.read_csv('data/PIRAMIDA PENDUDUK - Laki-laki.csv')
# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_lk.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_lk.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_lk = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Laki-laki"
)

kelompok_umur_pr = pl.read_csv('data/PIRAMIDA PENDUDUK - Perempuan.csv')
# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_pr.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_pr.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_pr = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Perempuan"
)

df_horizontal_join = melted_result_pr.join(melted_result_lk, on="Age_Group", how="inner")

# Daftar kategori usia
ku = ["0 - 1", "2 - 4", "5 - 9", "10 - 14", "15 - 19", 
      "20 - 24", "25 - 29", "30 - 34", "35 - 39", "40 - 44", 
      "45 - 49", "50 - 54", "55 - 59", "60 - 64", 
      "65 - 69", "70 - 74", "75+"]

# Menghitung berapa kali daftar ku perlu diulang
repeat_count = df_horizontal_join.shape[0] // len(ku) + 1
repeated_ku = (ku * repeat_count)[:df_horizontal_join.shape[0]]

# Menambahkan kolom kategori umur
df_horizontal_join = df_horizontal_join.with_columns([pl.Series(name="Kategori_Umur", values=repeated_ku)])

y_age = df_horizontal_join['Kategori_Umur'] 
x_M = df_horizontal_join['Laki-laki'] 
x_F = df_horizontal_join['Perempuan'] * -1

if max(x_M) >= max(x_F):
    maks = max(x_M)
else:
    maks = max(x_F)

import math

def round_up_to_nearest(number, base):
    return base * math.ceil(number / base)

def auto_round_up(number):
    if number == 0:
        return 0
    base = 10 ** (len(str(number)) - 1)
    return round_up_to_nearest(number, base)

maks1 = auto_round_up(maks)
maks2 = auto_round_up(int(maks - (maks * 1 / 3)))
maks3 = auto_round_up(int(maks - (maks * 2 / 3)))

tick_vals = [-maks1, -maks2, -maks3, 0, maks1, maks2, maks3]
tick_str = [str(abs(value)) for value in tick_vals]

import plotly.graph_objects as gp
# Creating instance of the figure 
fig = gp.Figure() 
  

# Adding Female data to the figure 
fig.add_trace(gp.Bar(y = y_age, x = x_F, 
                     name = 'Perempuan', orientation = 'h',
                     marker=dict(color='#ffc107'),
                     hovertemplate='Perempuan Umur %{y}<br>Jumlah: %{customdata}<extra></extra>',
                     customdata=[abs(x) for x in x_F]
                     )) 

# Adding Male data to the figure 
fig.add_trace(gp.Bar(y= y_age, x = x_M,  
                     name = 'Laki-laki',  
                     orientation = 'h',
                     marker=dict(color='#0d6efd'),
                     hovertemplate='Laki-laki Umur %{y}<br> %{x}<extra></extra>')) 

  
# Updating the layout for our graph 
fig.update_layout(title = 'Piramida Penduduk', 
                 title_font_size = 22, barmode = 'relative', 
                 bargap = 0.0, bargroupgap = 0, 
                 xaxis = dict(tickvals = tick_vals, 
                              ticktext = tick_str, 
                              title = 'Jumlah', 
                              title_font_size = 14),
                legend=dict(
                        orientation='h',
                        yanchor='bottom',
                        y=-0.3,  # Adjust this value to move the legend up or down
                        xanchor='center',
                        x=0.5
                ),
                plot_bgcolor='#f6f8fa',  # Set background color of the plot area to green
                paper_bgcolor='#f6f8fa'  # Set background color of the entire canvas to green 
)
  
fig.show()

In [56]:
df_horizontal_join.select('Kategori_Umur', 'Perempuan', 'Laki-laki')

Kategori_Umur,Perempuan,Laki-laki
str,i64,i64
"""0 - 1""",7111,7655
"""2 - 4""",28925,31161
"""5 - 9""",55267,59619
"""10 - 14""",58918,62885
"""15 - 19""",62332,67733
"""20 - 24""",60634,63400
"""25 - 29""",49743,50836
"""30 - 34""",44449,44164
"""35 - 39""",44751,43901
